# PROJEKT

## Co nas interesuje?

Głównie notatniki examine_model i augmentation_samples.
Zawartość pierwszego z nich jest poniżej w zmodyfikowanej formie i daje czytelny schemat działania całego programu.

Jako "model" autorzy rozumieją nakładkę klasyfikującą feature'y wychodzące z resneta/alexneta etc. Są nimi zapisane w models.py pipeliny klasyfikujące np. FV-SVC.
Model klasyfikujący jest optymalizowany przez gridsearch w hyperparameters.py na danych z feature_matrix.npy który wychodzi z extract_features.py.

Samo extract_features.py używa compute_feature_matrix() pochodzącej z features.py. Wewnątrz wspomnianej funkcji kluczowa jest  extract_features(), która faktycznie zwraca N wektorów w latencie (dla batcha obrazków rozmiaru N).


## Czego potrzeba na dniach 8-13.05

Raczej nie ma co liczyć, że będziemy mieli nasze rozwiązania wimplementowane w strukturę całego projektu, więc ograniczymy się do pusuzczenia działającego runa (parę zrobionych w augmentation_samples patchy -> VIT -> DS-MIL / FV-RF -> wynik) w tym notatniku.
Piętro niżej jest już zaimportowany vit_l_32. Natywne funkcje projektu nie współpracują z importowanym vitem. 

Potrzeba nam: 
- Kilka patchy zrobionych w augmentation_samples downscalowanych do 224x224 (vim przyjmuje (N, 3, 224, 224))
- Gotowych do klasyfikacji DS-MIL i FV-RF jako .pkl 
(aktualne modele nauczone są na innych rozmiarach wejść "3D tensor (N, W * H, C)", podczas gdy vim_l_32 wypuszcza (N, 1024))

In [1]:
from torchvision import models
vit = models.vit_b_16(pretrained=True)

ModuleNotFoundError: No module named 'torchvision'

In [2]:
conda install torchvision

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                       -                                                                         failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - alabaster -> python[version='>=2.7,<2.8.0a0|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.5,<3.6.0a0']
  - appdirs -> python[version='>=2.7,<2.8.0a0|>=3.5,<3.6.0a0|>=3.7,<3.8.0a0|>=3.6,<3.7.0a0']
  - argon2-cffi -> python[version='>=2.7,<2.8.0a0|>=3.11,<3.12.0a0|>=3.5,<3.6.0a0']
  - argon2

In [ ]:
from sklearn.externals import joblib
from skimage.io import imread
from matplotlib import pyplot as plt
from pipeline import features
import numpy as np
from torch import nn
from torchvision import models
from torchvision.transforms import Compose
from dataset.normalization import get_normalization_transform
from util.augmentation import NumpyToTensor
from pathlib import Path
from dataset import FungusDataset

In [ ]:
# you may need to adjust this accroding to your setup
model_path = './best_model.pkl' #FV-SVC, potrzeba nam jednego z DS-MIL i jednego FV-RF
patch_path = './patch.png'      #Potrzebujemy jednego patcha z augmentation_samples.ipynb
normalization_params_path = Path('.')

In [ ]:
# load and inspect model and image patch
model = joblib.load(model_path)
print(model)
patch = imread(patch_path)
plt.imshow(patch)

In [ ]:
# normalize image
transform = [
    NumpyToTensor(),
    get_normalization_transform(normalization_params_path),
]
transform = Compose(transform)
patch = transform(patch)

In [ ]:
# extract features from patch
device = features.get_cuda()
patch = patch.unsqueeze(0)
print(patch.shape)
# best model uses alexnet, so let's load it
# extractor = models.alexnet(pretrained=True).features.eval().to(device)
# features_matrix = features.extract_features(patch, device, extractor)
# features_matrix = features_matrix.detach().numpy()

# Używają alexneta. Implementuję w jego miejsce VIT'a vit_l_32 https://pytorch.org/vision/main/models/vision_transformer.html
vit = models.vit_l_32(pretrained=True)
vit.heads = nn.Identity() # Usuwanie klasyfikacji końcowej
features_matrix = vit(patch) 
features_matrix = features_matrix.detach().numpy()
print(features_matrix.shape)
# Wynik rozmiaru Nx1024 gdzie N to ilość patchy.

In [ ]:
# predict classification
result = model.predict(features_matrix)
FungusDataset.NUMBER_TO_FUNGUS[result[0]]